In [ ]:
!pip install rdkit
!pip install torch-geometric

In [2]:
import os
import glob
from rdkit import Chem
from rdkit.Chem import AllChem
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected

In [ ]:
!wget https://pdbbind.oss-cn-hangzhou.aliyuncs.com/download/PDBbind_v2020_refined.tar.gz

--2023-04-01 14:40:04--  https://pdbbind.oss-cn-hangzhou.aliyuncs.com/download/PDBbind_v2020_refined.tar.gz
Resolving pdbbind.oss-cn-hangzhou.aliyuncs.com (pdbbind.oss-cn-hangzhou.aliyuncs.com)... 121.199.204.182
Connecting to pdbbind.oss-cn-hangzhou.aliyuncs.com (pdbbind.oss-cn-hangzhou.aliyuncs.com)|121.199.204.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 690895072 (659M) [application/gzip]
Saving to: ‘PDBbind_v2020_refined.tar.gz.1’

PDBbind_v2020_refin 100%[===================>] 658.89M  8.16MB/s    in 90s     

2023-04-01 14:41:37 (7.29 MB/s) - ‘PDBbind_v2020_refined.tar.gz.1’ saved [690895072/690895072]



In [ ]:
!mkdir PDBbind_v2020_refined

In [ ]:
!tar -xzvf "PDBbind_v2020_refined.tar.gz" -C "PDBbind_v2020_refined"

In [ ]:
ls /content/PDBbind_v2020_refined/refined-set | wc -l

5318


In [ ]:
!rm -r /content/PDBbind_v2020_refined/refined-set

In [ ]:
%cd ~/../content/

/content


In [ ]:
!ls

Berberin.mol2  INDEX_refined_data.2020	PDBbind_v2020_refined.tar.gz
CDK1.pdb       PDBbind_v2020_refined	sample_data


In [39]:
import re

# 1. Прочитайте файл и извлеките информацию о связывающей аффинности для каждого идентификатора PDB
affinity_data_file = "INDEX_refined_data.2020"
affinity_data = {}
pdbs = list()
#data_folder = '/content/PDBbind_v2020_refined/refined-set'


with open(affinity_data_file, "r") as f:
    lines = f.readlines()
    for line in lines:
      if 'Kd=' in line:
        tokens = line.split()
        for token in tokens:
              if 'Kd' in token:
                value, unit = re.findall(r"[-+]?\d*\.\d+|\d+", token)[0], token[-2]
                value = float(value)
                  
                if unit == 'u':
                    value *= 1000  # convert from uM to nM
                elif unit == 'm':
                    value *= 1000000  # convert from mM to nM
                elif unit == 'p':
                    value *= 0.001  # convert from mM to nM
                  
                pdb_id = line.split()[0]
                affinity = value
                affinity_data[pdb_id] = value
                pdbs.append(pdb_id)

# 2. Сопоставьте идентификаторы PDB с соответствующими значениями связывающей аффинности
binding_affinities = []
for pdb_id in pdbs:
    if pdb_id in affinity_data:
        binding_affinities.append((pdb_id, affinity_data[pdb_id]))
    else:
        print(f"No affinity data found for {pdb_id}")

binding_affinities_dict = {pdb_id: affinity for pdb_id, affinity in binding_affinities}

In [145]:
print(binding_affinities_dict)
print(len(binding_affinities_dict))
values = list(binding_affinities_dict.values())
print(values)
#разыне значения um, микро, наномоли

{'2r58': 0.9900990098994216, '3c2f': 0.9999999999999999, '4qsu': 0.9900990098994216, '4qsv': 0.9900990098994216, '4u54': 0.8613861385919027, '3ao4': 0.841584158390746, '4cs9': 0.792079207887854, '2w8w': 0.7425742573849622, '6r9u': 0.7425742573849622, '5cs3': 0.6930693068820704, '5fe6': 0.6841584157915498, '6ghj': 0.6821782177714341, '4y3j': 0.6732673266809136, '5oxk': 0.6752475247010292, '4ahr': 0.6435643563791784, '4ahs': 0.6336633662786001, '4mre': 0.6336633662786001, '1x8d': 0.609900990037212, '4g0z': 0.6069306930070385, '2aac': 0.5940594058762866, '4aci': 0.5940594058762866, '4ury': 0.5940594058762866, '3ao5': 0.5841584157757082, '5m9w': 0.5643564355745515, '4np2': 0.5445544553733948, '1uto': 0.5267326731923536, '4ahu': 0.5346534652728163, '4ddk': 0.5049504949710812, '4g0y': 0.494059405860445, '5mz8': 0.4851485147699245, '5aol': 0.4752475246693461, '4y3y': 0.445544554367611, '5fsc': 0.4356435642670326, '6rnt': 0.4257425741664542, '3f6e': 0.40990099000552876, '4u5s': 0.3861386137641

In [53]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()

values = list(binding_affinities_dict.values())
values_np = np.array(values).reshape(-1,1)

scaled_values = scaler.fit_transform(values_np)

for i, key in enumerate(binding_affinities_dict):
    binding_affinities_dict[key] = scaled_values[i][0]

In [146]:
values = list(binding_affinities_dict.values())
print(values)

[0.9900990098994216, 0.9999999999999999, 0.9900990098994216, 0.9900990098994216, 0.8613861385919027, 0.841584158390746, 0.792079207887854, 0.7425742573849622, 0.7425742573849622, 0.6930693068820704, 0.6841584157915498, 0.6821782177714341, 0.6732673266809136, 0.6752475247010292, 0.6435643563791784, 0.6336633662786001, 0.6336633662786001, 0.609900990037212, 0.6069306930070385, 0.5940594058762866, 0.5940594058762866, 0.5940594058762866, 0.5841584157757082, 0.5643564355745515, 0.5445544553733948, 0.5267326731923536, 0.5346534652728163, 0.5049504949710812, 0.494059405860445, 0.4851485147699245, 0.4752475246693461, 0.445544554367611, 0.4356435642670326, 0.4257425741664542, 0.40990099000552876, 0.3861386137641407, 0.37623762366356234, 0.36633663356298396, 0.35643564346240564, 0.34653465336182726, 0.3376237622713067, 0.3366336632612489, 0.3366336632612489, 0.3267326731606705, 0.3267326731606705, 0.31683168306009213, 0.2970297028589354, 0.2970297028589354, 0.29207920780864616, 0.277227722657778

In [19]:
def mol_to_graph(mol):
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append(atom.GetAtomicNum())
    atom_features = torch.tensor(atom_features, dtype=torch.float).view(-1, 1)

    bond_features = []
    edge_index = []
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index.extend([[i, j], [j, i]])
        bond_features.extend([bond.GetBondTypeAsDouble()] * 2)
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    bond_features = torch.tensor(bond_features, dtype=torch.float).view(-1, 1)

    graph = Data(x=atom_features, edge_index=edge_index, edge_attr=bond_features)
    return graph

def read_receptor_pdb(path):
    mol = Chem.MolFromPDBFile(path)
    if mol is None:
        raise ValueError(f"Failed to read PDB file: {path}")
    return mol

def read_ligand_mol2(path):
    mol = Chem.MolFromMol2File(path)
    if mol is None:
        raise ValueError(f"Failed to read MOL2 file: {path}")
    return mol

#data_folder = '/content/PDBbind_v2020_refined/refined-set'
#graph_data = []
#skipped_files = 0
#counter = 0

#for pdb_id in os.listdir(data_folder):
#    receptor_path = os.path.join(data_folder, pdb_id, f"{pdb_id}_protein.pdb")
#    ligand_path = os.path.join(data_folder, pdb_id, f"{pdb_id}_ligand.mol2")

#    if os.path.isfile(receptor_path) and os.path.isfile(ligand_path):
#        try:
#            receptor_mol = read_receptor_pdb(receptor_path)
#            ligand_mol = read_ligand_mol2(ligand_path)
#
#            receptor_graph = mol_to_graph(receptor_mol)
#            ligand_graph = mol_to_graph(ligand_mol)

#            graph_data.append((pdb_id, receptor_graph, ligand_graph))
#            counter += 1
#            print(f"Processing: {counter}")
#        except Exception as e:
#            print(f"Error processing {pdb_id}: {e}")
#            skipped_files += 1

#print(f"Processed {len(graph_data)} receptor-ligand pairs.")
#print(f"Skipped {skipped_files} due to errors.")

In [147]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GraphModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, p=0.7, training=self.training)
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = global_mean_pool(x, data.batch)
        x = self.fc(x)
        return x

class BindingAffinityModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BindingAffinityModel, self).__init__()
        self.receptor_model = GraphModel(input_dim, hidden_dim, output_dim)
        self.ligand_model = GraphModel(input_dim, hidden_dim, output_dim)
        self.fc = nn.Linear(2 * output_dim, 1)

    def forward(self, receptor_data, ligand_data):
        receptor_embedding = self.receptor_model(receptor_data)
        ligand_embedding = self.ligand_model(ligand_data)
        x = torch.cat([receptor_embedding, ligand_embedding], dim=1)
        x = self.fc(x)
        return x

input_dim = 1
hidden_dim = 64
output_dim = 128
model = BindingAffinityModel(input_dim, hidden_dim, output_dim)

In [142]:
import torch.optim as optim
from torch_geometric.data import DataLoader
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data


graph_data_list = []

for i, (pdb_id, receptor_graph, ligand_graph) in enumerate(graph_data):
    receptor_data = Data(x=receptor_graph['x'], edge_index=receptor_graph['edge_index'], edge_attr=receptor_graph['edge_attr'])
    ligand_data = Data(x=ligand_graph['x'], edge_index=ligand_graph['edge_index'], edge_attr=ligand_graph['edge_attr'])
    
    if pdb_id in binding_affinities_dict:
      binding_affinity = torch.tensor([binding_affinities_dict[pdb_id]], dtype=torch.float32)

    data = {
        'receptor_graph': receptor_data,
        'ligand_graph': ligand_data,
        'binding_affinity': binding_affinity
    }

    graph_data_list.append(data)


# 1. Разделите датасет на обучающую и тестовую выборки
train_data, test_data = train_test_split(graph_data_list, test_size=0.2, random_state=42)

# 2. Создайте DataLoader для обучающей и тестовой выборок
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

#batch_size = 32
#data_loader = DataLoader(graph_data_list, batch_size=batch_size, shuffle=True)



# 3. Задайте функцию потерь и оптимизатор
criterion = nn.L1Loss()
optimizer = optim.RAdam(model.parameters(), lr=0.0001)

# 4. Обучите модель на обучающей выборке
num_epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        receptor_data, ligand_data = batch['receptor_graph'].to(device), batch['ligand_graph'].to(device)
        target = batch['binding_affinity'].to(device)

        optimizer.zero_grad()
        output = model(receptor_data, ligand_data)
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

    model.eval()
    test_loss = 0

    with torch.no_grad():
        for batch in test_loader:
            receptor_data, ligand_data = batch['receptor_graph'].to(device), batch['ligand_graph'].to(device)
            target = batch['binding_affinity'].to(device)

            output = model(receptor_data, ligand_data)
            loss = criterion(output.squeeze(), target)

            test_loss += loss.item()

    print(f"Test Loss: {test_loss / len(test_loader)}")

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([30, 1])) that is different to the input size (torch.Size([30])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 0.045438772268794676
Epoch 2, Loss: 0.0425863244092545
Epoch 3, Loss: 0.041200148596151454
Epoch 4, Loss: 0.040789585598316545
Epoch 5, Loss: 0.04060265470950587
Epoch 6, Loss: 0.04059553737877994
Epoch 7, Loss: 0.04009709265586492
Epoch 8, Loss: 0.040290309128829756
Epoch 9, Loss: 0.039819319845756165
Epoch 10, Loss: 0.03969721333753016
Epoch 11, Loss: 0.03921872841798373
Epoch 12, Loss: 0.03926607809105032
Epoch 13, Loss: 0.0384750868873419
Epoch 14, Loss: 0.038528399982464476
Epoch 15, Loss: 0.038258770833144315
Epoch 16, Loss: 0.03775948895191824
Epoch 17, Loss: 0.03748823511036667
Epoch 18, Loss: 0.03728741253852039
Epoch 19, Loss: 0.03670136591520261
Epoch 20, Loss: 0.03702044095592322
Epoch 21, Loss: 0.036278123073783274
Epoch 22, Loss: 0.035740402627837016
Epoch 23, Loss: 0.03576695447082858
Epoch 24, Loss: 0.035424847306834685
Epoch 25, Loss: 0.03525081613554141
Epoch 26, Loss: 0.03524502729241912
Epoch 27, Loss: 0.03452969133551862
Epoch 28, Loss: 0.03413174112

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([15, 1])) that is different to the input size (torch.Size([15])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [143]:
# Загрузите файлы рецептора и лиганда
receptor_path = "BACE1.pdb"
ligand_path = "Berberin.mol2"

# Преобразуйте файлы в графовое представление
receptor_mol = read_receptor_pdb(receptor_path)
ligand_mol = read_ligand_mol2(ligand_path)

receptor_graph = mol_to_graph(receptor_mol)
ligand_graph = mol_to_graph(ligand_mol)

# Создайте объекты Data для рецептора и лиганда
receptor_data = Data(x=receptor_graph['x'], edge_index=receptor_graph['edge_index'], edge_attr=receptor_graph['edge_attr'])
ligand_data = Data(x=ligand_graph['x'], edge_index=ligand_graph['edge_index'], edge_attr=ligand_graph['edge_attr'])

# Включите режим оценки для модели
#model.eval()

# Выполните прямой проход с данными рецептора и лиганда
#with torch.no_grad():
#    receptor_data = receptor_data.to(device)
#    ligand_data = ligand_data.to(device)
#    output = model(receptor_data, ligand_data)

# Интерпретируйте результаты
#predicted_binding_affinity = output.item()
#print(f"Predicted binding affinity: {predicted_binding_affinity}")

In [136]:
# Включите режим оценки для модели
model.eval()

# Выполните прямой проход с данными рецептора и лиганда
with torch.no_grad():
    receptor_data = receptor_data.to(device)
    ligand_data = ligand_data.to(device)
    output = model(receptor_data, ligand_data)

# Интерпретируйте результаты
predicted_binding_affinity = output.item()
print(f"Predicted binding affinity: {predicted_binding_affinity}")

Predicted binding affinity: 0.11705906689167023


In [144]:
# Включите режим оценки для модели
model.eval()

# Выполните прямой проход с данными рецептора и лиганда
with torch.no_grad():
    receptor_data = receptor_data.to(device)
    ligand_data = ligand_data.to(device)
    output = model(receptor_data, ligand_data)

# Интерпретируйте результаты
predicted_binding_affinity = output.item()
predicted_binding_affinity = scaler.inverse_transform(np.array(predicted_binding_affinity).reshape(-1,1))
predicted_binding_affinity = predicted_binding_affinity[0][0]
print(f"Predicted binding affinity: {predicted_binding_affinity}")

Predicted binding affinity: -60203.62439730446


In [7]:
import pickle

#with open('model.pickle', 'wb') as f:
#    pickle.dump(model, f)

In [ ]:
with open('model.pickle', 'rb') as f:
    model = pickle.load(f)

In [5]:
# Задайте путь для сохранения graph_data_list
graph_data_list_path = "/content/drive/MyDrive/graph_data.pkl"

# Сохраните graph_data_list на диск
#with open(graph_data_list_path, 'wb') as file:
#    pickle.dump(graph_data, file)

In [8]:
# Загрузите graph_data_list с диска
with open(graph_data_list_path, 'rb') as file:
    graph_data = pickle.load(file)
